# Big data analytics data preparation

This file details the data preparation process for one of my master's projects (the details can be found on my [website](www.jamiefawcett.org.uk).


In [ ]:
import pandas as pd
import numpy as np

## Loading data

In [ ]:
all_data = pd.read_csv("Full_2019-03-10.csv",index_col=0)

In [ ]:
display(all_data.head())
print(len(all_data))

## Cleaning up the data

In [24]:
#reset index
all_data = all_data.reset_index()

In [63]:
#making numeric
def makeInt(x):
    if x == None:
        return None
    if type(x) == str:
        x = x.replace(",","")
    return int(x)

In [59]:
for var in ["topic_reply_count", "topic_view_count", "forum_topic_count", "forum_post_count"]: ###posttypeid -- also a number but its a category -- choose not to make integer
    all_data[var] = all_data[var].map(lambda num: makeInt(num))

In [38]:
#fixing time

#fix today and yesterday
def makeDates(x):
    if "Today" in x:
        x = x.replace("Today","2019-03-10")
        
    if "Yesterday" in x:
        x = x.replace("Yesterday","2019-03-09")
    x = datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
    return x

In [38]:
#convert to datetime objects
all_data["time"] = all_data["time"].map(lambda date: makeDates(date))

In [54]:
def getAdditional(x,key):
    adit_dict = ast.literal_eval(x)
    if key in adit_dict:
        value = adit_dict[key]
        value = value.strip()
    else:
        value = None
    return value

In [54]:
keys = ['role', 'from', 'registr', 'num_post']

for key in keys:
    all_data[key] = all_data["additional"].map(lambda x: getAdditional(x,key))
    
all_data['num_post'] = all_data['num_post'].map(lambda num: makeInt(num))

## Aggregating data

In [5]:
#number of posts
topic_posts_author = all_data.groupby(['topic_title','forum_title','topic_link'])[['index','author']].nunique()
#views
topic_views = pd.Series(all_data.groupby(['topic_title','forum_title','topic_link'])['topic_view_count'].unique())
#replies
topic_replies = pd.Series(all_data.groupby(['topic_title','forum_title','topic_link'])['topic_reply_count'].unique())
#timestamps
post_timestamps = pd.Series(all_data.groupby(['topic_title','forum_title','topic_link'])['time'].unique())

In [6]:
topic_df = pd.concat([topic_posts_author,topic_views,topic_replies,post_timestamps], axis =1, sort= False)

In [7]:
#rename index as post
topic_df = topic_df.rename(columns={'index': 'posts'})

#unpack the topic_view count
topic_df['topic_view_count'] = topic_df['topic_view_count'].map(lambda l: l[0])
# topic_df['topic_view_count_unpack_num'] = topic_df['topic_view_count'].map(lambda l: len(l)) #check if any two still named the same

topic_df['topic_reply_count'] = topic_df['topic_reply_count'].map(lambda l: l[0])

In [8]:
#make time again
def makeTime(x):
    x = datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
    return x

topic_df['real_time'] = topic_df['time'].map(lambda time_list: [makeTime(time) for time in time_list])

In [10]:
#average time between replies
def getAverageTime(time_list):
    if len(time_list) == 1:
        av_time = 0
    else:
        av_time = np.diff(time_list).mean()
    return pd.to_timedelta(av_time)

#average time between replies
def getFirstResponse(time_list):
    if len(time_list) == 1:
        av_time = 0
    else:
        av_time = np.diff(time_list)[0]
    return pd.to_timedelta(av_time)

In [11]:
#get first day, last day and two averages
topic_df['first_post'] = topic_df['real_time'].map(lambda time_list: min(time_list))
topic_df['last_post'] = topic_df['real_time'].map(lambda time_list: max(time_list))
topic_df['conversation_length'] = topic_df.apply(lambda row: row['last_post']-row['first_post'],axis=1)
topic_df['total_average_response'] = topic_df.apply(lambda row: row['conversation_length']/row['posts'],axis=1) #kinda hacky
topic_df['running_average_response'] = topic_df['real_time'].map(lambda time_list: getAverageTime(time_list))
topic_df['first_response_time'] = topic_df['real_time'].map(lambda time_list: getFirstResponse(time_list))

In [15]:
# time to seconds

#Intervals to seconds   
interval_cols =['conversation_length', 'total_average_response',"running_average_response",'first_response_time']
for column in interval_cols:
    print(type(topic_df[column][0]))
    topic_df["seconds_{}".format(column)] = topic_df[column].map(lambda x: x.total_seconds())
    print(type(topic_df["seconds_{}".format(column)][0]))

<class 'pandas._libs.tslibs.timedeltas.Timedelta'>
<class 'numpy.float64'>
<class 'pandas._libs.tslibs.timedeltas.Timedelta'>
<class 'numpy.float64'>
<class 'pandas._libs.tslibs.timedeltas.Timedelta'>
<class 'numpy.float64'>
<class 'pandas._libs.tslibs.timedeltas.Timedelta'>
<class 'numpy.float64'>


In [16]:
def dt2ut(dt):
    epoch = pd.to_datetime('1970-01-01 00:00:00')
    return (dt - epoch).total_seconds()

#Dates ready for R
date_cols = ['first_post', 'last_post']
for column in date_cols:
    print(type(topic_df[column][0]))
    topic_df["unix_{}".format(column)] = topic_df[column].apply(dt2ut)
    print(type(topic_df["unix_{}".format(column)][0]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'numpy.float64'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'numpy.float64'>


In [17]:
display(topic_df.head())

,,,posts,author,topic_view_count,topic_reply_count,time,real_time,first_post,last_post,conversation_length,total_average_response,running_average_response,first_response_time,seconds_conversation_length,seconds_total_average_response,seconds_running_average_response,seconds_first_response_time,unix_first_post,unix_last_post
topic_title,forum_title,topic_link,,,,,,,,,,,,,,,,,,
!!! Карта для поиска пропавшего ребёнка Вытегра Вологодская область,users: Russia,viewtopic.php?id=16035,87,20,32632,86,"[2012-03-28 13:19:44, 2012-03-28 14:03:26, 201...","[2012-03-28 13:19:44, 2012-03-28 14:03:26, 201...",2012-03-28 13:19:44,2012-04-17 16:04:54,20 days 02:45:10,05:32:55.977011,05:36:48.255814,00:43:42,1737910.0,19975.977011,20208.255814,2622.0,1.332941e+09,1.334679e+09
!ANFÄNGERFRAGE! Wie eigene Karten wie beim Erdbeben in Nepal erstellen,users: Germany,viewtopic.php?id=31696,17,8,3208,16,"[2015-06-30 13:08:39, 2015-06-30 13:31:28, 201...","[2015-06-30 13:08:39, 2015-06-30 13:31:28, 201...",2015-06-30 13:08:39,2015-07-09 21:57:45,9 days 08:49:06,13:13:28.588235,14:03:04.125000,00:22:49,809346.0,47608.588235,50584.125000,1369.0,1.435670e+09,1.436479e+09
!i! in der Zeitung,users: Germany,viewtopic.php?id=8980,10,7,2044,9,"[2010-08-26 11:03:13, 2010-08-26 11:20:07, 201...","[2010-08-26 11:03:13, 2010-08-26 11:20:07, 201...",2010-08-26 11:03:13,2010-08-27 07:26:01,0 days 20:22:48,02:02:16.800000,02:15:52,00:16:54,73368.0,7336.800000,8152.000000,1014.0,1.282821e+09,1.282894e+09
""" Sehr zugewachsen "" man weiß wenigstens was einen erwartet",users: Germany,viewtopic.php?id=17131,9,7,1908,8,"[2012-06-22 19:15:10, 2012-06-22 19:18:08, 201...","[2012-06-22 19:15:10, 2012-06-22 19:18:08, 201...",2012-06-22 19:15:10,2012-06-23 20:30:13,1 days 01:15:03,02:48:20.333333,03:09:22.875000,00:02:58,90903.0,10100.333333,11362.875000,178.0,1.340393e+09,1.340483e+09
""" not found."" beim start von OSRM",users: Germany,viewtopic.php?id=25855,8,3,1909,7,"[2014-06-12 21:58:35, 2014-06-12 22:09:46, 201...","[2014-06-12 21:58:35, 2014-06-12 22:09:46, 201...",2014-06-12 21:58:35,2014-06-12 22:55:27,0 days 00:56:52,00:07:06.500000,00:08:07.428571,00:11:11,3412.0,426.500000,487.428571,671.0,1.402610e+09,1.402614e+09


In [18]:
topic_df.to_csv("Data-2019-03-10/Topic_2019-03-10.csv", mode ='w')

In [20]:
#For R
for col in ['first_post', 'last_post','conversation_length', 'total_average_response',"running_average_response",'first_response_time','time','real_time']:
    del topic_df[col]
topic_df.to_csv("Data-2019-03-10/Topic_BDA_2019-03-10.csv", mode ='w')